In [0]:
from google.colab import drive

drive.mount('/content/drive/')
%cd "/content/drive/My Drive/630Project"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/630Project


In [0]:
fh = open('seq2seq_summaries.txt')
seq = []
read_lines = fh.readlines()
for line in read_lines:
    seq.append(line)
fh.close()

fh = open('pre-trained-predictions.txt')
pointer = []
read_lines = fh.readlines()
for line in read_lines:
    pointer.append(line)
fh.close()

fh = open('extractive_summaries.txt')
ext = []
read_lines = fh.readlines()
for line in read_lines:
    ext.append(line)
fh.close()

fh = open('y_test_full.txt')
gold = []
read_lines = fh.readlines()
for line in read_lines:
    gold.append(line)
fh.close()

fh = open('X_test_full.txt')
articles = []
read_lines = fh.readlines()
for line in read_lines:
    articles.append(line)
fh.close()

In [0]:
!pip3 install rouge-score

In [0]:
from rouge_score import rouge_scorer

In [0]:
seq = [x.replace('#', '') for x in seq]
seq = [x.replace('<unk>', '') for x in seq]
seq = [x.replace('<end>', '') for x in seq]

pointer = [x.replace('<t>', '') for x in pointer]
pointer = [x.replace('</t>', '') for x in pointer]

In [0]:
rouge_precisions_seq1 = []
rouge_precisions_seq2 = []
rouge_precisions_seqL = []

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
i=0
for summary in gold:
  score = scorer.score(summary, seq[i])
  rouge_precisions_seq1.append(score['rouge1'][0])
  rouge_precisions_seq2.append(score['rouge2'][0])
  rouge_precisions_seqL.append(score['rougeL'][0])
  i+=1


rouge_precisions_pointer1 = []
rouge_precisions_pointer2 = []
rouge_precisions_pointerL = []

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
i=0
for summary in gold:
  score = scorer.score(summary, pointer[i])
  rouge_precisions_pointer1.append(score['rouge1'][0])
  rouge_precisions_pointer2.append(score['rouge2'][0])
  rouge_precisions_pointerL.append(score['rougeL'][0])
  i+=1



rouge_precisions_ext1 = []
rouge_precisions_ext2 = []
rouge_precisions_extL = []

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
i=0
for summary in gold:
  score = scorer.score(summary, ext[i])
  rouge_precisions_ext1.append(score['rouge1'][0])
  rouge_precisions_ext2.append(score['rouge2'][0])
  rouge_precisions_extL.append(score['rougeL'][0])
  i+=1

In [0]:
print("TextRank Scores")
print("ROUGE-1: " + str(sum(rouge_precisions_ext1)/len(rouge_precisions_ext1)))
print("ROUGE-2: " + str(sum(rouge_precisions_ext2)/len(rouge_precisions_ext2)))
print("ROUGE-L: " + str(sum(rouge_precisions_extL)/len(rouge_precisions_extL)))
print()

print("seq2seq Scores")
print("ROUGE-1 scores: " + str(sum(rouge_precisions_seq1)/len(rouge_precisions_seq1)))
print("ROUGE-2 scores: " + str(sum(rouge_precisions_seq2)/len(rouge_precisions_seq2)))
print("ROUGE-L scores: " + str(sum(rouge_precisions_seqL)/len(rouge_precisions_seqL)))
print()

print("Pointer-generator Scores")
print("ROUGE-1: " + str(sum(rouge_precisions_pointer1)/len(rouge_precisions_pointer1)))
print("ROUGE-2: " + str(sum(rouge_precisions_pointer2)/len(rouge_precisions_pointer2)))
print("ROUGE-L: " + str(sum(rouge_precisions_pointerL)/len(rouge_precisions_pointerL)))

TextRank Scores
ROUGE-1: 0.30447718537402096
ROUGE-2: 0.11899023009509402
ROUGE-L: 0.20376692942051178

seq2seq Scores
ROUGE-1 scores: 0.1562085815612292
ROUGE-2 scores: 0.06664159594732004
ROUGE-L scores: 0.1365092860068261

Pointer-generator Scores
ROUGE-1: 0.3211355425252225
ROUGE-2: 0.13736667248123224
ROUGE-L: 0.22639189237252635


In [0]:
!pip install textatistic
from textatistic import Textatistic

In [0]:
!pip install textstat
import textstat

In [0]:
# Original readability

flesch = []
ari = []

for x in articles:
  ari.append(textstat.coleman_liau_index(x))
  try:
    s = Textatistic(x)
    flesch.append(s.fleschkincaid_score)
  except:
    pass


print("average CL of original: " + str(sum(ari)/len(ari)))
t = sum(flesch)/len(flesch)
print("average flesch of original: " + str(t))

average CL of original: 11.657945859872582
average flesch of original: 11.360372737548175


In [0]:
# Average CL for each method -- focused on length as a metric for complexity

point = []
extract = []
sequence = []

for x in range(len(gold)):
  point.append(textstat.coleman_liau_index(pointer[x]))
  extract.append(textstat.coleman_liau_index(ext[x]))
  sequence.append(textstat.coleman_liau_index(seq[x]))

print("average CL of extractive: " + str(sum(extract)/len(extract)))
print("average CL of seq2seq: " + str(sum(sequence)/len(sequence)))
print("average CL of pointer-generator: " + str(sum(point)/len(point)))

average CL of extractive: 11.274228430804872
average CL of seq2seq: 9.81592646207299
average CL of pointer-generator: 10.067033873769574


In [0]:
# Average Flesch for ech method

point = []
extract = []
sequence = []

for x in range(len(gold)):
  try:
    s = Textatistic(pointer[x])
    point.append(s.fleschkincaid_score)
  except:
    pass

for x in range(len(gold)):
  try:
    s = Textatistic(ext[x])
    extract.append(s.fleschkincaid_score)
  except:
    pass

for x in range(len(gold)):
  try:
    s = Textatistic(seq[x])
    sequence.append(s.fleschkincaid_score)
  except:
    pass

print("average flesch_reading_ease of extractive: " + str(sum(extract)/len(extract)))
print("average flesch_reading_ease of seq2seq: " + str(sum(sequence)/len(sequence)))
print("average flesch_reading_ease of pointer-generator: " + str(sum(point)/len(point)))

average flesch_reading_ease of extractive: 11.542319513281303
average flesch_reading_ease of seq2seq: 8.843992109866592
average flesch_reading_ease of pointer-generator: 7.671086626555519
